In [9]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Define the scope of the API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# Load credentials from the JSON file
creds = Credentials.from_service_account_file('strange-anchor-430022-c5-5f84172fa08f.json', scopes=SCOPES)

# Authorize the gspread client
gspread_client = gspread.authorize(creds)

# Authorize the Drive API client
drive_service = build('drive', 'v3', credentials=creds)

In [ ]:
def find_or_create_folder(drive_service, folder_name):
    """Finds or creates a Google Drive folder."""
    folder_id = None
    query = "mimeType = 'application/vnd.google-apps.folder' and name = '{}'".format(folder_name)
    results = drive_service.files().list(q=query).execute()
    items = results.get('files', [])

    if items:
        folder_id = items[0]['id']
    else:
        # Create the folder if it doesn't exist
        file_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        }
        file = drive_service.files().create(body=file_metadata, fields='id').execute()
        folder_id = file.get('id')
        print(f"Folder '{folder_name}' created with ID: {folder_id}")

    return folder_id

def upload_dataframe_to_sheet(gspread_client, drive_service, df, sheet_name, folder_id):
    """Uploads a Pandas DataFrame to a new Google Sheet."""
    try:
        # Create a new spreadsheet using gspread
        spreadsheet = gspread_client.create(sheet_name)

        # Move the spreadsheet to the specified folder using Drive API
        # Remove the current parent (root) using removeParents
        drive_service.files().update(
            fileId=spreadsheet.id,
            removeParents='root'  # remove from root
        ).execute()

        # Add the new parent using addParents
        drive_service.files().update(
            fileId=spreadsheet.id,
            addParents=folder_id
        ).execute()

        # Get the first sheet
        worksheet = spreadsheet.sheet1

        # Convert DataFrame to list of lists
        data = [df.columns.values.tolist()] + df.values.tolist()

        # Update the sheet with the data
        worksheet.update('A1', data)
        print(f"DataFrame uploaded to Google Sheet '{sheet_name}' in folder ID: {folder_id}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example Usage
folder_name = 'My Pandas Sheets'
folder_id = find_or_create_folder(drive_service, folder_name)

data = {'col1': [1, 2, 3], 'col2': ['A', 'B', 'C']}
df = pd.DataFrame(data)

sheet_name = 'My DataFrame Sheet'
upload_dataframe_to_sheet(gspread_client, drive_service, df, sheet_name, folder_id)

Folder 'My Pandas Sheets' created with ID: 1PPW85EF-lJTgzX3oJk2oknixyHgitKo6
An error occurred: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/1YwQeFKvESPE-cHSdzKqW2sggAIi-GSGDEkg3TiNnmMQ?alt=json returned "The parents field is not directly writable in update requests. Use the addParents and removeParents parameters instead.". Details: "[{'message': 'The parents field is not directly writable in update requests. Use the addParents and removeParents parameters instead.', 'domain': 'global', 'reason': 'fieldNotWritable'}]">


In [8]:
def find_or_create_folder(client, folder_name):
    """Finds or creates a Google Drive folder."""
    folder_id = None
    query = "mimeType = 'application/vnd.google-apps.folder' and name = '{}'".format(folder_name)
    results = client.drive_service.files().list(q=query).execute()
    items = results.get('files', [])

    if items:
        folder_id = items[0]['id']
    else:
        # Create the folder if it doesn't exist
        file_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        }
        file = client.drive_service.files().create(body=file_metadata, fields='id').execute()
        folder_id = file.get('id')
        print(f"Folder '{folder_name}' created with ID: {folder_id}")

    return folder_id

folder_name = 'rula_case_data_export_demo'  # Replace with your desired folder name
folder_id = find_or_create_folder(client, folder_name)

AttributeError: 'Client' object has no attribute 'drive_service'